In [1]:
import spacy
from datasets import load_dataset

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the promptset dataset
promptset = load_dataset("pisterlabs/promptset")

##### Filter out non-english prompts and prompts outside the character range

In [2]:
from ftlangdetect import detect

def eng_only(prompt):
    lang = detect(prompt.strip().split("\n")[0])["lang"]

    return lang == "en"

# iterate all prompts
prompts = [x for prompt_list in promptset["train"]["prompts"] for x in prompt_list]
prompts = list(set(list(filter(lambda x: len(x) > 20 and len(x) < 200 and eng_only(x), prompts))))
print(len(prompts))

28143


##### Filter out prompts with more than one interpolation

In [3]:
import re

interpolations = []
one_interpolations = []
for p in prompts:
    variables = re.findall(r'{[^}]*}', p)
    if variables or "PLACEHOLDER" in p:
        interpolations.append(p)
        
    if len(variables) == 1 or p.count("PLACEHOLDER") == 1:
        one_interpolations.append(p)
        
len(interpolations), len(one_interpolations)

(12275, 7530)

##### Applying a Mood filter

In [48]:

# Function to detect if a sentence is interrogative
def is_interrogative(sent):
    """
    Note: Expects nlp object
    
    # WH-words (WDT, WP, WP$, WRB)
    WDT: Wh-determiner. This tag is used for wh-words that function as determiners, such as "which" or "what" in the phrase "What book are you reading?"
    WP: Wh-pronoun. This tag is for wh-pronouns, such as "who", "whom", or "whose". These words are used for asking questions about people.
    WP$: Possessive wh-pronoun. This tag is similar to WP but is used for the possessive form. An example would be "whose" in "Whose are these keys?"
    WRB: Wh-adverb. This tag is for wh-adverbs, like "where", "when", "why", and "how". These are used for asking questions about time, place, reason, or manner.
    
    # Auxliary verbs (MD, VBZ, VBP)
    MD: Modal verb (e.g., can, could, might, should). Modal verbs are often used in questions.
    VBZ: Verb, 3rd person singular present (e.g., does, is). This form is used in questions like "Does he run?" or "Is it big?"
    VBP: Verb, non-3rd person singular present (e.g., do, are). This form is used in questions like "Do you run?" or "Are they big?"
    """ 
    # Check if the first token is a WH-word or if the sentence ends with a question mark
    if sent[0].tag_ in ["WDT", "WP", "WP$", "WRB"] or sent.text.endswith("?"):
        return True
    # Check if there's an auxiliary verb at the beginning (simple heuristic)
    if len(sent) > 1 and sent[1].tag_ in ["MD", "VBZ", "VBP"] and sent[0].dep_ == "aux":
        return True
    return False


def is_imperative(sentence):
    """
    Note: Expects nlp object
    """
    imperative_keywords = ['please']
    
    # Check for sentences starting with "Let's" as a common imperative form
    # if sentence.text.lower().startswith("let's"):
    #     return True
    
    # Check if the first word is an imperative keyword or an adverb
    if sentence[0].lemma_.lower() in imperative_keywords or sentence[0].pos_ == "ADV":        
        # Check if the next token is a verb
        for i in range(1, len(sentence)):
            if sentence[i].pos_ != "PUNCT":
                # Check if the first non-punctuation token is a verb
                if sentence[i].pos_ == "VERB":
                    return True
                break  # Stop searching once the first non-punctuation token is found

    
    # Auxiliary verbs check
    if sentence[0].pos_ == "VERB" and sentence[0].dep_ != "aux":
        return True
    
    # TODO: Add more rules for detecting imperatives (e.g., "Don't", etc.)
    return False

def is_valid_mood(text):
    # Process the text
    doc = nlp(text)
    # Check each sentence in the document
    for sent in doc.sents:
        if is_interrogative(sent) or is_imperative(sent):
            return True
        
    return False

# For testing purposes
assert is_valid_mood("What is the capital of {TEXT}?")
assert is_valid_mood("Write a sentence about {PLACEHOLDER}.")
assert is_valid_mood("Is this an example of an interrogative sentence?")
assert is_valid_mood("Did he win? The race is over.")
assert is_valid_mood("Please, go.")
assert is_valid_mood("Never give up!")
assert is_valid_mood("Let's do it!")
assert is_valid_mood("Always attend!")
assert is_valid_mood("Do it now!")

from tqdm import tqdm
filtered_prompts = []
for p in tqdm(one_interpolations):
    if is_valid_mood(p):
        filtered_prompts.append(p)

100%|██████████| 7530/7530 [00:38<00:00, 196.53it/s]


In [49]:
len(filtered_prompts)

3310

In [50]:
filtered_prompts[:10]

['What is a good name for a company that make {product}?',
 'when was the {summaryofbook} published',
 'Describe following file content: PLACEHOLDER',
 'Answer the question: "{query}". Explain reasoning and make sure to give your final answer in () parentheses',
 'Answer the question based on the pdf text given earlier. PLACEHOLDER',
 'Write a blog article in the format of the given outline\n                    Outline:\n                    {outline}',
 'Translate this email from {language} to English. Here is the email:\nPLACEHOLDER',
 'Review this legal document excerpt for any elements that seem legally unfair under US law "PLACEHOLDER"',
 '\n        Your mission is convert database result to meaningful sentences. Here is the database result: {database_result}. validate the query before responding.\n        ',
 'Summarize the following text:\n\n{}']

##### Shuffle the prompts (seed=42)

In [54]:
# shuffle the filtered prompts with seed 42
import random
random.seed(42)

shuffled_prompts = filtered_prompts.copy()
random.shuffle(shuffled_prompts)
shuffled_prompts[:100]

['Create a vector embedding for the following case information: PLACEHOLDER',
 'Pleaes analaze the following function, symbol or system calls that were pulled from the file? PLACEHOLDER',
 "Also consider {name}'s core characteristics given the",
 "Assume that there is/are SQL table(s) named 'PLACEHOLDER' ",
 'Determine the probability of the patient to have {topic}',
 'What country is {city} located in?',
 'given the full name {name_of_person}, I want you to find the linkedin profile page of the person\n    Your answer should only content the URL',
 'Answer like the rapper drake.PLACEHOLDER',
 'paraphrase the following text in an ELI5 style:\n{answer}',
 'Help summarize the article.: PLACEHOLDER',
 'Generate professional summary based on PLACEHOLDER.\n\n',
 "Some invalid data is provided. Provide the details to the consumer as depicted in json in natural language, don't display in json format\nPLACEHOLDER",
 'respond with a verbal backchannel to "{}"',
 'Convert the following verbal de

In [52]:
def check_and_reformat(prompt):
    """
    Checks if prompt is valid. If prompt is valid, returns a slightly modified prompt that can be evaluated and optimized.
    """
    pattern1 = r"{[^}]*}"
    pattern2 = "PLACEHOLDER"
    matches1 = re.findall(pattern1, prompt)
    condition1 = len(matches1) == 1 
    condition2 = prompt.count(pattern2) == 1
    
    if not condition1 and not condition2:
        print(prompt)
    
    # Reformat the prompt
    if condition1:
        return prompt.replace(matches1[0], "{TEXT}")
    elif condition2:
        return prompt.replace(pattern2, "{TEXT}")
    
    raise ValueError("Invalid prompt format. Prompt must contain some str/var to be interpolated.")


new_prompts = []
for p in filtered_prompts:
    try:
        new_prompts.append(check_and_reformat(p))
    except Exception as e:
        print(e)
        print(f"Skiiping prompt\n\n")

In [53]:
new_prompts

['What is a good name for a company that make {TEXT}?',
 'when was the {TEXT} published',
 'Describe following file content: {TEXT}',
 'Answer the question: "{TEXT}". Explain reasoning and make sure to give your final answer in () parentheses',
 'Answer the question based on the pdf text given earlier. {TEXT}',
 'Write a blog article in the format of the given outline\n                    Outline:\n                    {TEXT}',
 'Translate this email from {TEXT} to English. Here is the email:\nPLACEHOLDER',
 'Review this legal document excerpt for any elements that seem legally unfair under US law "{TEXT}"',
 '\n        Your mission is convert database result to meaningful sentences. Here is the database result: {TEXT}. validate the query before responding.\n        ',
 'Summarize the following text:\n\n{TEXT}',
 'Read the claim and the summaries of three articles below. Select the article that best matches the claim in essence.\n\nClaim: {TEXT}\n\nArticles:\n',
 'In short bullet points

##### Rest is Cross-Encoder stuff. 

In [ ]:

from sentence_transformers import CrossEncoder
from itertools import product
import torch
import json


PROMPT_LIST_ERR = [
    'Please format the following raw transcript for readability, including punctuation, speaker labels (look for semicolons after names), and spacing. Remove filler words:\n\nPLACEHOLDER\n',
    'You are a helpful assistant for Aidan. Your task is to correct any spelling discrepancies in the transcribed text. Only add necessary punctuation such as periods, commas, and capitalization, and use only the context provided. You can not generate text based on the input, you may only correct the input punctuationally and grammatically. If the transcribed text is blank then do not return anything\n\nPLACEHOLDER',
    "Correct the grammar in the sentence: PLACEHOLDER",
    'Reformat the following transcript into Markdown, bolding the speakers. Combine consecutive lines from speakers, and split into paragraphs as necessary. Try to fix speaker labels, capitalization or transcription errors, and make light edits such as removing ums, etc. There is some Danish, please italicize the Danish sentences. Reply with only the corrected transcript as we will be using your output programmatically:\n\nPLACEHOLDER',
    "Please fix the grammatical errors in this English translation of Bhagavad Gita. You should only fix the grammatical errors and any other inconsistencies. Do not change the meaning.\n\nPLACEHOLDER",
    "\n\nHuman: Here is an article, contained in <article> tags:\n\n            <article>\n            {input}\n            </article>\n\n            Please identify any grammatical errors in the article. Also, add the fixed article at the end of answer.\n            \n            Assistant: ",
    "rewrite my message, correct the grammar and make it more friendly, natural, shorter, and clearer. PLACEHOLDER",
    "Correct any grammar mistakes in the following text and return the corrected text: PLACEHOLDER",
    "\n                         Proof read this 'PLACEHOLDER',\n                         and correct any spelling or grammar mistakes.\n                        ",
    "Generate a grammar correction of the following sentence:\n\nPLACEHOLDER",
    'You are mainly an english teacher named Mr.Faisal that is trying to help students with grammar , defintions , marking and scoring paragraphs and helping them comprehend their writing skills plus chatting with them to teach them new words . allow questions in arabic about english and answer them in arabic , if they try to go of topic tell them Sorry student but I can only help with English PLACEHOLDER',
    "\n\tCorrect any grammatical, spelling errors in the question below. \n\tOutput only the corrected version and nothing else\n\tQuestion: {question}\n\tCorrected version: \n\t",
    "Please improve the following text by fixing grammar, spelling, and style:\n\nPLACEHOLDER",
    "You are given some input sentences. Fix the grammar and write the grammatical sentences.\n\ninputs: PLACEHOLDER\n\noutputs:\n",
    'Objective: To enhance official documents written. \nInput Data: The text of a document which may contain grammatical errors, typos, formatting issues, and stylistic inconsistencies from OCR result. \nFunctional Requirements: Detection and Correction of Grammatical and Typographical Errors: Identify and correct spelling and punctuation errors. Check grammatical agreements within sentences.\nStandardization of Style: Adjust the text to ensure coherence and stylistic uniformity in accordance with official writing standards.\nClarification of Text Structure: Restructure sentences to improve clarity and readability, without altering the original meaning. Keep and answer the detected language from the document.\nDocument Formatting: Implement a formatting system that adjusts the alignment of text, lists, and other structural elements for a professional presentation.\nOutput Data: This is the corrected and enhanced document. Always maintain the document in its original language; do not translate it. Respond only in the language detected from the document. Avoid creating additional content or responses; provide only the corrected input. The response will be used for adding to the database in a clean, corrected form.\nThe text: {text}. ',
    "Please rewrite the following text for more clarity and make it grammatically correct. Give me the updated text. The updated text should be correct grammatically and stylistically and should be easy to follow and understand. Only make a change if it's needed. Try to follow the style of the original text. Don't make it too formal. Include only improved text no other commentary.\n\nThe text to check:\n---\nPLACEHOLDER\n---\n\nImproved text: ",
    'Please rephrase the following question into good grammar.\nPlease respond in same language.\n\nQuestion:\n{question}\n\nRephrased question:',
    "Correct the grammar: PLACEHOLDER",
    # NOTE: Newly Added Prompts
    '\n  Analyse the given sentence relating to a land sale and see what words are wrong and out of place(these may sometimes include names and jibberish characters) in it and remove them, also correct grammar mistakes and make the sentence logically correct\n  In effect clean the sentence preserving all the pertinent and important details. By no means add fake details.:" {text}"\n  Response:\n  ',
    '\n  Proofread and correct the following text\n  and rewrite the corrected version. If you don\'t find\n  any errors, just say "No errors found":\n  ```PLACEHOLDER```\n  ',  # partial error correction
    ' Gramatically and logically correct sentence: {prompt_source} . Return only the corrected sentence, no abbreviations, using same words if it is logical. Do not mention explicitly rules given in prompt. ',
    'G\n\n                For Every Image , refine the text extracted from every image and if there is any error or spelling mistake , correct it by your knowledge  but correct only \'words\' and\n\n\'not\' values of it .\n\nAlso give "quantity" with it\'s value as it is given in extracted text, "rate" with it\'s value as it is given in extracted text and ("amount = quantity*rate") in the last as note. Give nothing else other than this \nPLACEHOLDER\n',  # I don't understand this prompt at all. haha...
    'Is the following text grammatically correct? Ignore style and meaning, just strictly check grammar.\nIf it is grammatically correct, just respond with \'yes\', nothing more.\nIf it is grammatically incorrect, respond with the corrected text. Do not explain what you did or why, just correct it.\n\nText:\n"""\n{text}\n"""\nResponse:\n',  # partial error correction
    'Proofread and correct the following text\n    and rewrite the corrected version. If you don\'t find\n    and errors, just say "No errors found". Don\'t use\n    any punctuation around the text:\n    ```PLACEHOLDER```',
    'Proofread and correct the following text\n    and rewrite the corrected version. If you don\'t find\n    and errors, just say "No errors found". Don\'t use \n    any punctuation around the text:\n    ```PLACEHOLDER```',  # the difference is a whitespace. hmmmmmmm
    'Proofread and correct the following text \n        and rewrite the corrected version. Only output the corrected version. Do not add any other words. \n        ```PLACEHOLDER```',
    'Proofread and correct the following text and rewrite the corrected version. If you don\'t find and errors, just say "No errors found". Don\'t use any punctuation around the text: ```PLACEHOLDER```',
    # # NOTE: Newly Added Prompts with no interpolations
    # '\n        You are a text processor and translator, correcting text and translating to english. Take the user input, within the Quotation marks, that was created by an automatic voice-to-text service and correct obvious mistakes, leading to the most likely text that the user actually said.\n        Do not change the meaning of the sentence. Only look for spelling mistakes and grammar errors and translate to english. If there are no obvious errors within the text, reply with the unchanged text in english. Always translate to english.\n        {TEXT}',  # partial error correction
    # 'You are an editor for a blog. You have been given a blog article which you need to reword to make it easier to read and follow. Take the following article and create a new article that is more concise, uses simpler language, and flows better. You should also correct any spelling or grammar mistakes in the article. Do not respond with anything except the corrected article. {TEXT}',
    # '1. I want you to be a grammar checker similar to QuillBot and Grammarly.\n2. You should be capable of identifying and correcting grammatical errors in a given text.)\n3. You should provide suggestions for corrections, offering alternatives to improve sentence structure, word choice, and overall grammar.\n4. The output should maintain the context and meaning of the original text while addressing the identified errors.\n5. Produce only plain text output, without any additional elements.\n\ntext:\n{TEXT}',
    # 'Correct any grammar and spelling mistakes in the user dialogue transcript. {TEXT}',
    # 'You are a highly skilled language model AI that returns only one line of grammatically perfect text. Your task is to evaluate the text below and correct its grammar. Even if the text is incomplete or unintelligible, YOU MUST make a grammatical correction, you can make assumptions about the intended meaning. If the text is grammatically correct, do not change it. Your output should be presented WITH ONLY the corrected text IN ONE LINE and without any extra dialogue from you. Do not use any new lines in your output. Your output should only have one line. {TEXT}',  # partial error correction
    # '\n                    You are an ai assistant that helps students                     correct and practice their english writing,                     the following sentence has grammar mistakes,                     correct them and output only the corrected sentence, nothing else.\n                    {TEXT}',
    # "\n        You are a specialized text fixer for invoices. Your task is to:\n        - Correct any spelling or grammatical errors.\n        - Reorder any text that may be in the wrong sequence.\n        - Separate words that are combined and combine words that are split.\n        - Extract and organize only the essential structured data such as:\n            - Customer details (Name)\n            - Item lists (Description, Quantity, Price)\n            - Totals (Subtotal, Taxes, Final Total)\n        - Remove any redundant or unnecessary information.\n        - Return the cleaned and structured text in a way that's easy to read and analyze.\n    {TEXT}",  # partial error correction
    # "You are a highly skilled language model AI that returns only one line of linguistically diverse paraphrased text. Your task is to perform two specific actions on a given text. First, evaluate each text and make sure it's grammatically correct. If a text is not grammatically correct, fix it. Then, ALWAYS paraphrase the text while maintaining its original meaning. Your output should be presented WITH ONLY the paraphrased text IN ONE SINGLE LINE, without any extra dialouge from you. Do not use any new lines in your output. Only write in one line. {TEXT}",  # partial error correction
    # 'Fix the grammar in the following text: {TEXT}'    
]

PROMPT_LIST_TRANS = [
    'Please translate this Indonesian text "PLACEHOLDER" to english in the format [EN: translation], but if there is no English translation, return [EN: Cannot be translated]. Please make sure write in the format that I requested only.',
    'Please help me to translate the following text. Please return only translated content not include the origin text. Here is the text: \n\nPLACEHOLDER',
    "translate the following text into English:\nPLACEHOLDER",
    "Robot asked child: 'What would you like to translate and to which language do you want to translate it?'. The child replied: PLACEHOLDER. Give the response without asking follow-up questions.",
    'PLACEHOLDER \n Please translate the previous sentence into English',
    'Please translate the following English passage into Bengali. Ensure that the translation is accurate and retains the original meaning and tone of the passage. The passage reads: PLACEHOLDER',
    "translate 'PLACEHOLDER' to English, and just show the result only, no other words",
]

PROMPT_LIST_SUMM = [
    "You are an onboarding chatboat that's very friendly and methodical. You read PLACEHOLDER and summarise the current project",
    "PLACEHOLDER\n Can you summarize this GitHub Pull Request for me and suggest possible improvements?",
    'Human: Summarize the code below (enclosed in the <code> tags) and explain in bullet points what it does. Write the response in markdown format starting with `## Summary`\n\nCode to be summarized:\n<code>\n{code}\n</code>\n\nAssistant:\n',
    "Summarize the following text. Keep the original language in \nwhich the text is written. The summary has to be shorter than the original text. Don't add up or make up\nany information not present in the original text.\nText: {text}",
]

PROMPT_LIST_QA = [
    'Given that someone is feeling PLACEHOLDER, what would be a good course of action or suggestion for them?',
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:",
    'Explanation of what the code does:PLACEHOLDER',
    'what is the city {person} is from?',
    'Tell me what language this is: ```PLACEHOLDER```',
    'what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:',
    "'PLACEHOLDER'\n Return a valid python dictionary where each key is a stock/industry name (ticker) and the contents is a detailed explanation of what that the person said",
    'What is the capital of {place}?',
]

# target_prompts = [
#     "Proofread and correct the following text\n    and rewrite the corrected version. If you don't find\n    and errors, just say \"No errors found\". Don't use \n    any punctuation around the text:\n    ```PLACEHOLDER```",
#     "\n                    You are an ai assistant that helps students                     correct and practice their english writing,                     the following sentence has grammar mistakes,                     correct them and output only the corrected sentence, nothing else.\n                    ",
#     # "Passage: It charges fast as advertised. What really sells to me is the size. It is smaller than an iPad charger and charges faster than the MacBook charger. I can charge my headphones, my phone, my iPad and my laptop with a single charger, AND IT FITS. I am a student and I stay on campus in between lectures so not needing a brick for each device is great when I study all day. I definitely recommend this to anyone. I go out of my way to put my friends on it. I even bought one with an EU plug from amazon.nl so I can use it when I go back home. I was so fascinated by how well built it is and how good it works, I went back to repeat my electronics courses to see how it could be done. When I need a cable or an adapter I go to the Anker page before googling it. Even their power strips are my go to.\n\nTLDR: fast, small, built, fits\n--\nPassage: Looking for socks that are thin, loose fitting, thinner after the first wash and no thickness/bulk cushioning? If so, add to cart. If not, move on to another brand. I thought ordering the Hanes brand for the first time. they would be good quality. I was wrong!!!\n\nTLDR: thin, loose, cushionless\n--\nPassage: We use several of these at work, my mom and I also each have one at home. They are not consistent at all. One day #1 and #2 are completely different readings from each other, the next they're the same, but #3 is different and #4 says something completely different. #1 will give 4 different readings in a row varying over 1 degree one day, then the same reading 4x in a row the next day. I cannot trust them at all without using multiple ones and hoping at least 2 work that day. My mom's does a weird flashing thing even with new batteries. Mine usually reads differently than another model I have. They're also not super fast.\n\nTLDR: inconsistent, weird, slow\n--\nPassage: You get what you pay for. The mirror feels cheap and is very flimsy. The handle and bezels will most definitely snap and break apart if you are slightly too rough with it. The mirror itself however, I have no complaints about. It's well polished and clear. Perfect for everyday use, just be extra careful with it.\n\nTLDR: cheap, flimsy, polished\n--\nPassage: Love this keyboard, it is the best one I have ever used at this price point. I dislike mechanical keyboards and since they are kind of the standard today I have bought a lot of non-mechanical keyboards in the $20-$30 price range looking for one that I like. This one by far is the winner. It is sturdy, the smooth texture of the metal frame feels nice and is easy to keep clean. My only nit-pick is that the brightness adjustment of the LED does not go as low as I would like, the lowest setting is still a bit bright.\n\nTLDR: sturdy, smooth, easy\n--\nPassage: This is a great shirt, fits well and is very comfortable. However with my experience, the Jerzees brand does seem to run a bit small, unless you're one that prefers a snug fit. Personally, I like my shirts a little on the loose side. If you read some of the other reviews, I think you'll find that other people found that the sizing was not what they expected. So I would definitely order a size up from what you normally wear. I normally wear a large. I got this shirt in an extra large and it fits and feels great.\n\nTLDR: fits, comfortable\n--\nPassage: Disaster. Have tried to self-install, have our HVAC tech install, and no one can consistently get this thermostat to control our brand new HVAC system. It blows cold when the heat is on, blows nothing when the a/c is on. Amazon won\u2019t let me return because I\u2019m out of my window\u2026 and Google won\u2019t let me return because I bought on Amazon. SCAM.\n\nTLDR: disaster, useless, scam\n--\nPassage: I have been using Chapstick for a while without any issues. Yesterday night I tried the Burt's bees lip balm and then next morning, my lips got so irritated and peeled off compared to the previous days. Initially I wasn't sure what caused this, then after a while I realized that I applied a different lip balm and then came here to Amazon to see the reviews having irritated skin and lips getting more chapped. I thought that Burt's bees was a good brand. I am so disappointed with the results.\n\nTLDR: irritated, disappointed\n--\nPassage: Yeah too small\n\nTLDR: small\n--\nPassage: PLACEHOLDER\n\nTLDR:",
# ]

# prompt_list_err = list(
#     map(lambda x: x.strip().replace("PLACEHOLDER", "{input}"), PROMPT_LIST_ERR)
# )


ce = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    default_activation_function=torch.nn.Sigmoid(),
)

prompt_lists = list(product(PROMPT_LIST_QA, one_interpolations))
results = ce.predict(prompt_lists, show_progress_bar=True)  # NOTE: batch_size=32 is default

In [ ]:
# results_json = []
# for prompt, score in zip(prompts, results):
#     results_json.append({"prompt": prompt, "score": float(score)})


# with open("ce_results3.json", "w") as f:
#     json.dump(results_json, f, indent=2)

In [ ]:
len(results), len(prompt_lists), len(one_interpolations), len(prompts)

In [ ]:
prompt_lists[0]

In [ ]:
from collections import defaultdict
import json

p_scores = defaultdict(float)
for pair in zip(prompt_lists, results):
    prompt = pair[0][1]
    score = pair[1]
    p_scores[prompt] = max(score, p_scores[prompt])

# jsonify p_scores
results_json = []
for k,v in p_scores.items():
    results_json.append({"prompt": k, "score": float(v)})
with open("ce_results3.json", "w") as f:
    json.dump(results_json, f, indent=2)

In [ ]:
import json

with open("ce_results3.json", encoding="utf-8") as f:
    data = json.load(f)

ec_prompts = []
for point in data:
    if point["score"] > 0.98 and len(point["prompt"]) < 2000:
        ec_prompts.append(point["prompt"])

with open("ec_prompts.json", "w", encoding="utf-8") as f:
    json.dump(ec_prompts, f, indent=2, ensure_ascii=False)

In [ ]:
len(set(ec_prompts).intersection(set(one_interpolations)))

In [ ]:
len(set(ec_prompts).intersection(set(PROMPT_LIST_QA)))